Reference code 
Source: https://github.com/mrpickleapp/ray-tracer-v1

In [ ]:
%%writefile colour.py
class Colour():
    def __init__(self, r, g, b):
        self.r = r
        self.g = g
        self.b = b

    def getList(self):
        return [self.r, self.g, self.b]

    def addColour(self, colour):
        return Colour(self.r + colour.r, self.g + colour.g, self.b + colour.b)

    def scaleRGB(self, scale, return_type=None):
        if return_type == None:
            return Colour(self.r * scale, self.g * scale, self.b * scale)
        if return_type == 'list':
            return [round(self.r * scale), round(self.g * scale), round(self.b * scale)]
        if return_type == 'Colour':
            return Colour(round(self.r * scale), round(self.g * scale), round(self.b * scale))

    def illuminate(self, light):
        r_factor = light.r / 255
        g_factor = light.g / 255
        b_factor = light.b / 255
        return Colour(
            round(self.r * r_factor),
            round(self.g * g_factor),
            round(self.b * b_factor)
        )

In [ ]:
%%writefile light.py
from colour import Colour

def incidence(angle, max_angle):
    if angle > max_angle:
        return 0
    if angle == 0:
        return 1
    rel_strength = ((max_angle - angle) / max_angle)
    return rel_strength

class GlobalLight():
    def __init__(self, vector, colour, strength, max_angle, func=0):
        self.vector = vector         # angle light is coming from
        self.colour = colour
        self.strength = strength     # 0-1
        self.max_angle = max_angle   # the greatest angle light is reflected from - eg 90 degrees
        self.func = func             # 0: linear
    
    def relativeStrength(self, angle):
        if self.func == 0:
            return self.colour.scaleRGB(incidence(angle, self.max_angle) * self.strength)


class PointLight():
    def __init__(self, id, position, colour, strength, max_angle, func=0):
        self.id = id            # set id to object id if object is emitting light
        self.position = position     # point of origin
        self.colour = colour
        self.strength = strength     # 0-1
        self.max_angle = max_angle
        self.func = func                # 0: linear / inverse square rule

    def relativeStrength(self, angle, distance):
        if self.func == -1:
            return self.colour.scaleRGB(incidence(angle, self.max_angle) * self.strength)
        if self.func == 0:
            return self.colour.scaleRGB(incidence(angle, self.max_angle) * self.strength / distance)

In [ ]:
%%writefile material.py

class Material():
    def __init__(self, reflective=0, transparent=0, emitive=0, refractive_index=1):
        self.reflective = reflective        # 0-1
        self.transparent = transparent      # 0-1
        self.emitive = emitive              # 0-1
        self.refractive_index = refractive_index


    # Behaviour when hit by ray
        # could emit

        # could reflect

        # could absorb

        # could return value



matte = Material(

)

In [ ]:
%%writefile object.py
from colour import Colour

class Sphere():
    def __init__(self, centre, radius, material, colour=Colour(128, 128, 128), id=0):
        self.id = id
        self.centre = centre
        self.radius = radius
        self.material = material
        self.colour = colour

In [ ]:
%%writefile vector.py
import math
import numpy as np
from numpy import sin, cos, tan, arccos

class Vector:

    @staticmethod
    def fromNpArray(array):
        return Vector(x=array[0], y=array[1], z=array[2])

    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def describe(self, caption=""):
        print(f"{caption}x: {self.x}, y: {self.y}, z: {self.z}")

    def getXYZ(self):
        return self.x, self.y, self.z

    def toNpArray(self):
        return np.array([self.x, self.y, self.z])

    def addVector(self, B, inplace=False):
        if inplace == True:
            self.x += B.x
            self.y += B.y
            self.z += B.z
            return self
        return Vector(self.x + B.x, self.y + B.y, self.z + B.z)

    def subtractVector(self, B, inplace=False):
        if inplace == True:
            self.x -= B.x
            self.y -= B.y
            self.z -= B.z
            return self
        return Vector(self.x - B.x, self.y - B.y, self.z - B.z)

    def invert(self, inplace=False):
        if inplace == True:
            self.x = -self.x
            self.y = -self.y
            self.z = -self.z
            return self
        return Vector(-self.x, -self.y, -self.z)
    
    def scaleByLength(self, l, inplace=False):
        if inplace == True:
            self.x *= l
            self.y *= l
            self.z *= l
            return self
        else:
            return Vector(self.x * l, self.y * l, self.z * l)

    def distanceFrom(self, B):
        return math.sqrt((B.x - self.x)**2 + (B.y - self.y)**2 + (B.z - self.z)**2)

    def angleBetween(self, B):
        return arccos(self.dotProduct(B) / (self.magnitude() * B.magnitude()))

    def reflectInVector(self, B):
        v = self.normalise()
        normal = B.normalise()
        return v.subtractVector(normal.scaleByLength(2 * v.dotProduct(normal))).normalise()

    def refractInVector(self, B, r_index_a, r_index_b):

        # https://www.scratchapixel.com/lessons/3d-basic-rendering/introduction-to-shading/reflection-refraction-fresnel.html

        v = self.normalise()
        normal = B.normalise()
        
        n = r_index_a / r_index_b

        cosI = v.dotProduct(normal)
        if cosI < -1:
            cosI = -1
        if cosI > 1:
            cosI = 1

        if cosI < 0:
            cosI = -cosI
        
        k = 1 - n**2 * (1 - cosI**2)

        if k < 0:
            return False

        return v.scaleByLength(n).addVector(normal.scaleByLength(n * cosI - math.sqrt(k))).normalise()

    def dotProduct(self, B):
        return self.x * B.x + self.y * B.y + self.z * B.z

    def crossProduct(self, B):
        # denoted by A x B
        return Vector(
            x=self.y*B.z - self.z*B.y,
            y=self.z*B.x - self.x*B.z,
            z=self.x*B.y - self.y*B.x
        )

    def magnitude(self):
        # ||v|| denotes the length of a vector
        dotProduct = self.dotProduct(self)
        return math.sqrt(dotProduct)

    def normalise(self):
        magnitude = self.magnitude()
        return Vector(x=self.x/magnitude, y=self.y/magnitude, z=self.z/magnitude)

    def multiplyByMatrix(self, T):
        return self.fromNpArray(np.matmul(self.toNpArray(), T))

    def rotate(self, angle, inplace=False):
        a, b, c = angle.x, angle.y, angle.z
        R = np.array([
            [cos(c)*cos(b)*cos(a) - sin(c)*sin(a), cos(c)*cos(b)*sin(a) + sin(c)*cos(a), -cos(c)*sin(b)],
            [-sin(c)*cos(b)*cos(a) - cos(c)*sin(a), -sin(c)*cos(b)*sin(a) + cos(c)*cos(a), sin(c)*sin(b)],
            [sin(b)*cos(a), sin(b)*sin(a), cos(b)]
        ])
        V = np.matmul(np.array([self.x, self.y, self.z]), R)
        if inplace == True:
            self = Vector(x=V[0], y=V[1], z=V[2])
        return Vector(x=V[0], y=V[1], z=V[2])



class Angle:

    # x = rotation in the xy plane
    # y = rotation around the y axis (positive is left)
    # z = bank (positive is left)

    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

In [ ]:
%%writefile ray.py
import math
import numpy as np

from vector import Vector, Angle
from object import Sphere
from colour import Colour

class Intersection():

    @staticmethod
    def nearestIntersection(intersections):
        nearestIntersection = None
        for intersection in intersections:
            if intersection.intersects == True:
                if nearestIntersection == None:
                    nearestIntersection = intersection
                else:
                    if intersection.distance < nearestIntersection.distance:
                        nearestIntersection = intersection
        return nearestIntersection

    def __init__(self, intersects=False, distance=None, point=None, normal=None, object=None, bounces=0, through_count=0):
        self.intersects = intersects
        self.distance = distance
        self.point = point
        self.normal = normal
        self.object = object
        self.bounces = bounces
        self.through_count=through_count

    def directionRGB(self):
        # ray has not landed on anything, but might get light from light sources

        # test
        return Colour(0, 255,)

    def terminalRGB(self, spheres, background_colour=Colour(0, 0, 0), global_light_sources=[], point_light_sources=[], max_bounces=0):
        # colour of the thing landed on
        reflectivity, transparency, emitivity = self.object.material.reflective, self.object.material.transparent, self.object.material.emitive 
        
        illumination = self.object.colour.scaleRGB(emitivity)      # does not take distance from camera into account

        for light in global_light_sources:
            angle_to_light = self.normal.angleBetween(light.vector)
            illumination = illumination.addColour(light.relativeStrength(angle_to_light))
        
        for light in point_light_sources:
            if self.object.id != light.id:
                vector_to_light = light.position.subtractVector(self.point)
                ray_to_light = Ray(
                    origin=self.point,
                    D=vector_to_light
                )
                ray_to_light_terminus = ray_to_light.nearestSphereIntersect(spheres, suppress_ids=[self.object.id], max_bounces=max_bounces)

                if ray_to_light_terminus != None:
                    # clear line of sight
                    if ray_to_light_terminus.object.id == light.id:

                        angle_to_light = self.normal.angleBetween(vector_to_light)
                        distance_to_light = vector_to_light.magnitude()
                        illumination = illumination.addColour(light.relativeStrength(angle_to_light, distance_to_light))

        # resolve final total of illumination
        return background_colour.addColour(self.object.colour.illuminate(illumination))


class Ray():
    def __init__(self, origin, D):
        self.origin = origin
        self.D = D.normalise()  # direction in vector

    def sphereDiscriminant(self, sphere, point=0):      # set point to 1 when you want the second intersection
        O = self.origin
        D = self.D
        C = sphere.centre
        r = sphere.radius
        L = C.subtractVector(O)

        tca = L.dotProduct(D)
        if tca < 0:     # intersection is behind origin - this doesn't work when line is inside sphere
           return Intersection()

        d = None
        try:
            d = math.sqrt(L.dotProduct(L) - tca**2)
        except:
            d = 0       # crude error protection - in case D & L are too similar
        if d > r:       # line misses sphere
            return Intersection()

        thc = math.sqrt(r**2 - d**2)
        t0 = tca - thc      # distance to first intersection
        t1 = tca + thc      # distance to second intersection

        tmin = [t0, t1][point]

        phit = O.addVector(D.scaleByLength(tmin))     # point of intersection
        nhit = phit.subtractVector(C).normalise()     # normal of intersection

        return Intersection(
            intersects=True,
            distance = tmin,
            point = phit,
            normal = nhit,
            object = sphere
        )

    def sphereExitRay(self, sphere, intersection):

        # refract at first intersection
        refracted_ray_D = self.D.refractInVector(intersection.normal, 1, sphere.material.refractive_index)

        # get internal ray
        internal_ray = Ray(
            origin=intersection.point,
            D=refracted_ray_D
        )

        # get second intersection
        exit_intersection = internal_ray.sphereDiscriminant(sphere=sphere, point=1)

        exit_ray_D = None
        exit = False

        n = 0
        while (exit == False) & (n < 10):
            n+=1

            # refract exit ray
            exit_ray_D = refracted_ray_D.refractInVector(exit_intersection.normal.invert(), sphere.material.refractive_index, 1)

            if exit_ray_D != False:
                exit = True
            else:
                # TIR
                refracted_ray_D = refracted_ray_D.reflectInVector(exit_intersection.normal)
                # find next exit point
                exit_ray = Ray(
                    origin=exit_intersection.point,
                    D=refracted_ray_D
                )
                exit_intersection = exit_ray.sphereDiscriminant(sphere=sphere, point=1)
            
        if exit == True:

            return Ray(
                exit_intersection.point,
                exit_ray_D
            )

        # TRAPPED RAY:
        print("TRAPPED RAY:")
        self.origin.describe()
        self.D.describe()

        return None


    def nearestSphereIntersect(self, spheres, suppress_ids=[], bounces=0, max_bounces=1, through_count=0):

        intersections = []

        for i, sphere in enumerate(spheres):
            if sphere.id not in suppress_ids:
                intersections.append(self.sphereDiscriminant(sphere))

        nearestIntersection = Intersection.nearestIntersection(intersections)
        
        if nearestIntersection == None:
            return None

        if bounces > max_bounces:
            return None

        nearestIntersection.bounces = bounces
        nearestIntersection.through_count = through_count

        # NB - reflective objects return background colour if no reflections found
        if nearestIntersection.object.material.reflective == True:
            
            reflected_ray_D = self.D.reflectInVector(nearestIntersection.normal)
            
            reflected_ray = Ray(
                origin=nearestIntersection.point,
                D=reflected_ray_D
            )
            bounces += 1
            suppress_ids = [nearestIntersection.object.id]
            reflected_terminus = reflected_ray.nearestSphereIntersect(
                spheres=spheres,
                suppress_ids=suppress_ids,
                bounces=bounces,
                max_bounces=max_bounces,
                through_count=through_count
            )

            if reflected_terminus != None:
                return reflected_terminus
            
            return nearestIntersection

        # REFRACTION
        if nearestIntersection.object.material.transparent == True:

            sphere_exit_ray = self.sphereExitRay(
                sphere=nearestIntersection.object,
                intersection=nearestIntersection
            )

            if sphere_exit_ray == None:
                return None
            
            bounces += 1
            through_count += 1
            suppress_ids = [nearestIntersection.object.id]

            reflected_terminus = sphere_exit_ray.nearestSphereIntersect(
                spheres=spheres,
                suppress_ids=suppress_ids,
                bounces=bounces,
                max_bounces=max_bounces,
                through_count=through_count
            )

            if reflected_terminus != None:
                return reflected_terminus

            return None

        return nearestIntersection



In [ ]:
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from vector import Vector, Angle
from object import Sphere
from ray import Ray, Intersection
from material import Material
from colour import Colour
from light import GlobalLight, PointLight

base_material = Material(reflective=False)
reflective_material = Material(reflective=True)
glass = Material(reflective=False, transparent=True, refractive_index=1.52)
emitive_material = Material(emitive=True)

default_material = base_material

sphere_1 = Sphere(
    id=1,
    centre=Vector(-0.8, 0.6, 0),
    radius=0.3,
    material=glass,
    colour=Colour(255, 100, 100)
)
sphere_2 = Sphere(
    id=2,
    centre=Vector(0.8, -0.8, -10),
    radius=2.2,
    material=default_material,
    colour=Colour(204, 204, 255)
)
sphere_3 = Sphere(
    id=3,
    centre=Vector(0.3, 0.34, 0.1),
    radius=0.2,
    material=default_material,
    colour=Colour(0, 51, 204)
)
sphere_4 = Sphere(
    id=4,
    centre=Vector(5.6, 3, -2),
    radius=5,
    material=reflective_material,
    colour=Colour(153, 51, 153)
)
sphere_5 = Sphere(
    id=5,
    centre=Vector(-0.8, -0.8, -0.2),
    radius=0.25,
    material=default_material,
    colour=Colour(153, 204, 0)
)
sphere_6 = Sphere(
    id=6,
    centre=Vector(-3, 10, -75),
    radius=30,
    material=default_material,
    colour=Colour(255, 204, 102)
)

spheres = [sphere_6]

ray_origin = Vector(0, 0, 1)

rays = []

RAY_COUNT = 100   # = rays in each axis = 2 * ray count + 1 (for centre ray)
RAY_STEP = 0.01   # 0.003 = long focal distance

multiple = 3     # 1 is normal, 2 is ok, 5 is good, 10 is mega
RAY_COUNT *= multiple
RAY_STEP /= multiple

X_RAYS = [r*RAY_STEP for r in range(-RAY_COUNT, 0, 1)] + [r*RAY_STEP for r in range(0, RAY_COUNT + 1)]
Y_RAYS = [r*RAY_STEP for r in range(RAY_COUNT, 0, -1)] + [-r*RAY_STEP for r in range(0, RAY_COUNT + 1)]
Z = -1

MAX_BOUNCES = 5

TOTAL_RAYS = len(X_RAYS) * len(Y_RAYS)

print(f"Generating {TOTAL_RAYS} rays...")
for Y in Y_RAYS:
    for X in X_RAYS:
        ray = Ray(
            origin=ray_origin,
            D=Vector(x=X, y=Y, z=-1)
        )
        rays.append(ray)

lightVector = Vector(3, 1, -0.75)     # infinite distance light source

global_light_sources = []
global_light_sources.append(GlobalLight(vector=Vector(3, 1, -0.75), colour=Colour(20, 20, 255), strength=1, max_angle=np.radians(90), func=0))

point_light_sources = []
sun = Sphere(id=0, centre=Vector(-0.6, 0.2, 6), radius=0.1, material=emitive_material, colour=Colour(255, 255, 204))
sun_light = PointLight(id=sun.id, position=sun.centre, colour=sun.colour, strength=1, max_angle=np.radians(90), func=-1)
spheres.append(sun)
point_light_sources.append(sun_light)

max_angle = np.radians(90)
background_colour = Colour(2, 2, 5)
black_colour = Colour(0, 0, 0)

pixels = []

progress_milestones = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

for i, ray in enumerate(rays):

    progress = (i+1) / len(rays) * 100
    if progress > progress_milestones[0]:
        progress_milestones.pop(0)
        print(f"Casting Rays: {progress:.0f}%", end="\r")

    
    ray_terminal = ray.nearestSphereIntersect(spheres, max_bounces=MAX_BOUNCES)

    if ray_terminal == None:
        pixels.append(background_colour.getList())

    else:
        pixels.append(ray_terminal.terminalRGB(
            spheres=spheres,
            background_colour=background_colour,
            global_light_sources=global_light_sources,
            point_light_sources=point_light_sources
        ).getList())

X_SIZE = len(X_RAYS)
Y_SIZE = len(Y_RAYS)

pixels = np.array(pixels).reshape((X_SIZE, Y_SIZE, 3))

plt.figure(figsize=(6, 6))
plt.imshow(pixels)
plt.axis('off')
plt.savefig('render_output.png', bbox_inches='tight')
plt.show()